In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline


: 

In [ ]:
# read in all the words
words = open('names.txt, 'r).read().splitlines()
words[:3]